# Копируем данные и устанавливаем программы

Для корректности относительных путей следует запускать из папки data

## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
^C


In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [3]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 33.8 MB/s 


## Аннотация генома (пример)

Biopython Tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [16]:
!head -n 20 gms2.lst

# GeneMark.hmm-2 LST format
# GeneMark.hmm-2 prokaryotic version: 1.25_lic
# File with sequence: scaffolds.fasta
# File with native parameters: GMS2.mod
# Native species name and build: unspecified GeneMarkS-2-1.14_1.25_lic
# File with MetaGeneMark parameters: /content/gms2_linux_64/mgm_11.mod
# translation table: 11
# output date start: Thu Oct 20 21:18:59 2022

# sequence-region 1 3830280
SequenceID: scaffold1_len3830264_cov231
     1   -    77    535     459 native ATGGAA 6 1
     2   -    767    1585     819 native TTAGAG 6 1
     3   +    1727    3106    1380 native CTGGCC 7 1
     4   -    3133    3975     843 native GAGGAG 5 1
     5   -    4082    6781    2700 native GAGCAA 8 1
     6   -    6778    7557     780 native AAGGTC 11 1
     7   +    7851    8597     747 native AAGGTA 6 1
     8   +    8682    9536     855 native GAGGAA 7 1
     9   +    9645    10376     732 native TTGGAG 7 1


Для примера проаннотируем два коротких скаффолда (scaffold3_cov275 и scaffold9_cov256):



In [6]:
from Bio import SeqIO

record_data = {}
for record in SeqIO.parse("/content/scaffolds.fasta", "fasta"):
    record.annotations['molecule_type'] = 'DNA'
    record_data[record.id] = record

In [9]:
SeqIO.write(record_data.values(), "GENOME.gbk", "genbank")

/usr/local/lib/python3.7/dist-packages/Bio/SeqIO/InsdcIO.py:730: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  BiopythonWarning,


71

## Добавляем координаты генов

```
SequenceID: scaffold3_cov275
  3535   -    30    1001     972 atypical GTCGAG 5 1
  3536   -    1350    2192     843 native AAAGTG 7 1
  3537   +    2304    2507     204 native GCGGAG 7 1
  3538   -    2554    2751     198 native TAAGTA 7 1
  3539   +    2855    3265     411 native TCGGTC 6 1
```

In [77]:
import pandas as pd
import io

def parse_gene_start(string):
    return int(str(string).replace("<", ""))

def parse_gene_end(string):
    return int(str(string).replace(">", ""))

def parse_lst_file(filename):
    with open(filename, "r") as f:
        content = f.read()

    result = {}
    blocks = content.split("\n\n")
    for block_i, block in enumerate(blocks):
        lines = block.split("\n")
        if lines[0].startswith("# sequence-region"):
            sequence_id = lines[1].replace("SequenceID: ", "")
            table_lines = lines[2:-1]
            if len(table_lines) == 0:
                result[sequence_id] = None
                continue
            table_content = "\n".join(table_lines)
            df = pd.read_csv(io.StringIO(table_content), header=None, sep="\s+", names=['LocusTag', 'Strand', 'Start', 'End', 'Length', 'Type', 'Data1', 'Data2', 'Data3'])
            df["Start"] = df["Start"].apply(parse_gene_start)
            df["End"] = df["End"].apply(parse_gene_end)
            df["LocusTag"] = df["LocusTag"].apply(int)
            df["Strand"] = df["Strand"].apply(lambda string: 1 if string == "+" else -1)
            df["Length"] = df["Length"].apply(int)
            result[sequence_id] = df
    return result


In [82]:
lst_data = parse_lst_file("gms2.lst")

In [108]:
mil_data = pd.read_csv("scaffolds.hits_from_MIL_1.txt", sep="\s+", names="qseqid, sseqid, pident, length, mismatch, gapopen, qstart, qend, sstart, send, evalue, bitscore".split(", "))
mil_data["CCU"] = mil_data["sseqid"].apply(lambda string: string.split("_")[2] if string.split("_")[2].startswith("CCU") else None)

def get_mil_best_matching_protein(locus_tag):
    hits = mil_data[mil_data["qseqid"] == locus_tag]
    if hits.empty:
        return None
    return mil_data.iloc[hits['evalue'].idxmin()]["CCU"]

mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

mil_protein_data = {}
for mil_f in mil_1_genome.features:
    if 'protein_id' not in mil_f.qualifiers:
        continue
    mil_protein_data[mil_f.qualifiers['protein_id'][0]] = mil_f.qualifiers['product']

In [117]:
swissprot_data = pd.read_csv("scaffolds.hits_from_SwissProt.txt", sep="\s+", names="qseqid, sseqid, pident, length, mismatch, gapopen, qstart, qend, sstart, send, evalue, bitscore".split(", "))
swissprot_data["ProteinID"] = swissprot_data["sseqid"].apply(lambda string: string.split("|")[2])

def get_swissprot_best_matching_protein(locus_tag):
    hits = swissprot_data[swissprot_data["qseqid"] == locus_tag]
    if hits.empty:
        return None
    row = swissprot_data.iloc[hits['evalue'].idxmin()]
    return row["ProteinID"], row["evalue"]

In [119]:
from pprint import pprint
from Bio.SeqFeature import SeqFeature, FeatureLocation

# Аминокислотные последовательности
protein_data = {}
for record in SeqIO.parse("proteins.fasta", "fasta"):
    protein_data[int(record.id)] = record

class FeatureCreator:
    def __init__(self):
        self.genes_counter = 0
        self.genes_mil_counter = 0
        self.genes_swissprot_counter = 0
        self.genes_not_found = 0

    def create_feature_from_table_row(self, row):
        feature = SeqFeature(FeatureLocation(row["Start"], row["End"], strand=row["Strand"]), type="CDS")
        locus_tag = row["LocusTag"]
        feature.qualifiers['locus_tag'] = [locus_tag]

        self.genes_counter += 1

        if locus_tag in protein_data:
            protein_sequence = protein_data[locus_tag]
            feature.qualifiers['translation'] = [protein_sequence.seq]
        
        mil_best_ccu = get_mil_best_matching_protein(locus_tag)
        if mil_best_ccu:
            self.genes_mil_counter += 1
            feature.qualifiers['note'] = [mil_best_ccu]
            feature.qualifiers['product'] = mil_protein_data[mil_best_ccu]
        else:
            swiss_best_protein_id_e = get_swissprot_best_matching_protein(locus_tag)
            if swiss_best_protein_id_e:
                self.genes_swissprot_counter += 1
                feature.qualifiers['product'] = f'Similar to SwissProt protein {swiss_best_protein_id_e[0]} (E-value = {swiss_best_protein_id_e[1]})'
            else:
                self.genes_not_found += 1
        return feature

    def run(self):
        for record_id, record in record_data.items():
            record_genes = lst_data[record_id]
            if record_genes is None:
                continue
            
            features = record_genes.apply(self.create_feature_from_table_row, axis=1)
            record.features = list(features)
        SeqIO.write(record_data.values(), "GENOME.gbk", "genbank")
        print("Всего генов", self.genes_counter)
        print("Проаннотировали через родственную бактерию", self.genes_mil_counter)
        print("Проаннотировали через swissprot", self.genes_swissprot_counter)
        print("Не удалось проаннотировать", self.genes_not_found)
f = FeatureCreator()
f.run()

/usr/local/lib/python3.7/dist-packages/Bio/SeqIO/InsdcIO.py:730: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  BiopythonWarning,


Всего генов 3569
Проаннотировали через родственную бактерию 3275
Проаннотировали через swissprot 53
Не удалось проаннотировать 241


## Добавляем функции белков (из БД SwissProt)

Читаем информацию из файла scaffolds.hits_from_SwissProt.txt

In [113]:
!head scaffolds.hits_from_SwissProt.txt

197	sp|P44152|T1SH_HAEIN	30.400	250	153	6	39	282	29	263	2.07e-12	73.2
198	sp|O51040|Y007_BORBU	27.698	278	164	12	6	259	9	273	1.53e-17	83.6
253	sp|P39979|HPA3_YEAST	31.081	148	99	2	3	148	14	160	1.76e-12	64.3
656	sp|P27278|NADR_ECOLI	27.554	323	177	13	11	308	67	357	3.58e-15	79.7
656	sp|P24518|NADR_SALTY	27.864	323	176	13	11	308	67	357	4.19e-15	79.3
656	sp|P44308|NADR_HAEIN	25.806	310	200	9	11	308	61	352	1.57e-14	77.8
932	sp|P9WHG7|PARE1_MYCTU	36.957	92	58	0	2	93	3	94	7.55e-21	82.0
932	sp|P9WHG6|PARE1_MYCTO	36.957	92	58	0	2	93	3	94	7.55e-21	82.0
932	sp|Q9A9T8|PARE1_CAUVC	32.609	92	60	1	1	92	1	90	5.73e-16	69.7
933	sp|P58093|PARD_VIBCH	66.667	78	26	0	1	78	1	78	6.77e-32	108


product = 'Similar to SwissProt protein Y1178_METJA (E-value = 2.95e-28)'

Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [120]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2022-10-20 23:13:14--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  42.6MB/s    in 7.0s    

2022-10-20 23:13:21 (87.5 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [121]:
!gzip -d uniprot_sprot.dat.gz

In [122]:
!grep Q55EH8 uniprot_sprot.dat

AC   Q55EH8;
DR   AlphaFoldDB; Q55EH8; -.
DR   SMR; Q55EH8; -.
DR   PaxDb; Q55EH8; -.
DR   InParanoid; Q55EH8; -.
DR   PhylomeDB; Q55EH8; -.
DR   PRO; PR:Q55EH8; -.
